# Speech Recognition with Delta-Delta's 
This model will convert recorded speech data into Mel-Spectrogram and then to Delta and finally the Delta-Delta's. 
The data will then be put into a neural network 

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import librosa 
import librosa.display 


In [ ]:
# load the dataset that has file paths and labels for each observation 
df = pd.read_csv('/Users/stephen/Emotion_Dectection/data/RAVDESS/metadata.csv',index_col=0)
df.head()

In [ ]:
# for this model we'll drop some columns 
df.drop(columns=['modality', 'vocal_channel', 'emotional_intensity', 'statement', 'repetition', 'actor', 'actor_sex'], inplace=True)
df.sort_values(by='emotion')

In [ ]:
df['Path'] = df.index


In [ ]:
# https://librosa.org/doc/main/generated/librosa.feature.delta.html

def extract_features(data):
    sample_rate = 16000

    result = np.array([])

    # MFCC
    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mfcc)) # stacking horizontally
    
    # MFCC Delta
    mfcc_delta = librosa.feature.delta(mfcc)
    result = np.hstack((result, mfcc_delta)) # stacking horizontally

    # MFCC Delta-Delta
    mfcc_delta2 = librosa.feature.delta(mfcc, order=2)
    result = np.hstack((result, mfcc_delta2)) # stacking horizontally
    
    return result

def get_features(path):
    # duration and offset are used to take care of the no audio in start and the ending of each audio files as seen above.
    data, sample_rate = librosa.load(path, duration=2.5, offset=0.6)
    
    # without augmentation
    res1 = extract_features(data)
    result = np.array(res1)
    

    
    return result

In [112]:
X, Y = [], []
for path, emotion in zip(df.Path, df.emotion):
    feature = get_features(path)
    for ele in feature:
        X.append(ele)
        # appending emotion 3 times as we have made 3 augmentation techniques on each audio file.
        Y.append(emotion)

In [ ]:
len(X), len(Y), df.Path.shape

In [ ]:
Features = pd.DataFrame(X)
Features['labels'] = Y
Features.to_csv('features.csv', index=False)
Features.head()